In [22]:
import pandas as pd

In [23]:
mobile = pd.read_csv('./Mobile Classification.csv')
mobile.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [25]:
mobile.price_range.value_counts()

1    500
2    500
3    500
0    500
Name: price_range, dtype: int64

In [3]:
mobile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   battery_power  2000 non-null   int64  
 1   blue           2000 non-null   int64  
 2   clock_speed    2000 non-null   float64
 3   dual_sim       2000 non-null   int64  
 4   fc             2000 non-null   int64  
 5   four_g         2000 non-null   int64  
 6   int_memory     2000 non-null   int64  
 7   m_dep          2000 non-null   float64
 8   mobile_wt      2000 non-null   int64  
 9   n_cores        2000 non-null   int64  
 10  pc             2000 non-null   int64  
 11  px_height      2000 non-null   int64  
 12  px_width       2000 non-null   int64  
 13  ram            2000 non-null   int64  
 14  sc_h           2000 non-null   int64  
 15  sc_w           2000 non-null   int64  
 16  talk_time      2000 non-null   int64  
 17  three_g        2000 non-null   int64  
 18  touch_sc

In [4]:
mobile['price_range'].nunique()

4

In [5]:
mobile['price_range'].unique()

array([1, 2, 3, 0])

Assumption: If the price range is 0 the mobile is cheaper,and if the price range is 3 the price range is costlier.
To validate the above assumption we need to group the price ranges based on average ram.

In [6]:
mobile.groupby('price_range')['ram'].mean().reset_index()

,price_range,ram
0,0,785.314
1,1,1679.490
2,2,2582.816
3,3,3449.232


In [7]:
#Identifying the input and output variable
y = mobile[['price_range']] 
x = mobile.drop(columns =['price_range'])

In [8]:
x.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
0,842,0,2.2,0,1,0,7,0.6,188,2,2,20,756,2549,9,7,19,0,0,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,6,905,1988,2631,17,3,7,1,1,0
2,563,1,0.5,1,2,1,41,0.9,145,5,6,1263,1716,2603,11,2,9,1,1,0
3,615,1,2.5,0,0,0,10,0.8,131,6,9,1216,1786,2769,16,8,11,1,0,0
4,1821,1,1.2,0,13,1,44,0.6,141,2,14,1208,1212,1411,8,2,15,1,1,0


Splitting the data into train and test.

In [9]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,train_size = 0.8,random_state = 0)
len(x_train),len(x_test),len(y_train),len(y_test)

(1600, 400, 1600, 400)

Building the random forest

In [10]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=500)
model = rf.fit(x_train,y_train)


/var/folders/0n/72s_5dyn5gx6dwyk5y8sll780000gn/T/ipykernel_13119/296643341.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model = rf.fit(x_train,y_train)


In [11]:
#Predicting on test data
y_test['Prediction'] = model.predict(x_test)
y_test

,price_range,Prediction
405,3,3
1190,0,0
1132,2,2
731,2,1
1754,2,3
...,...,...
638,1,1
360,1,1
1810,0,0
1743,2,3


In [12]:
#Building the confusion and accuracy matrix
from sklearn.metrics import accuracy_score,confusion_matrix

In [13]:
confusion_matrix(y_test['price_range'],y_test['Prediction'])

array([[ 92,   3,   0,   0],
       [  5,  75,  12,   0],
       [  0,  18,  72,   9],
       [  0,   0,   8, 106]])

In [14]:
accuracy_score(y_test['price_range'],y_test['Prediction'])

0.8625

Lets use the model on a new data point.

In [15]:
x.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
0,842,0,2.2,0,1,0,7,0.6,188,2,2,20,756,2549,9,7,19,0,0,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,6,905,1988,2631,17,3,7,1,1,0
2,563,1,0.5,1,2,1,41,0.9,145,5,6,1263,1716,2603,11,2,9,1,1,0
3,615,1,2.5,0,0,0,10,0.8,131,6,9,1216,1786,2769,16,8,11,1,0,0
4,1821,1,1.2,0,13,1,44,0.6,141,2,14,1208,1212,1411,8,2,15,1,1,0


In [16]:
#A new mobile with the below features is coming up in the market
val_dict = {'battery_power':[700],
           'blue':[1],
           'clock_speed':[1.0],
           'dual_sim':[1],
           'fc':[0],
           'four_g':[1],
           'int_memory':[10],
           'm_dep':[0.5],
           'mobile_wt':[150],
           'n_cores':[6],
           'pc':[10],
           'px_height':[1200],
           'px_width':[1800],
           'ram':[3000],
           'sc_h':[15],
           'sc_w':[7],
           'talk_time':[15],
           'three_g':[0],
           'touch_screen':[0],
           'wifi':[0]}

In [17]:
val_df = pd.DataFrame(val_dict)
val_df

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
0,700,1,1.0,1,0,1,10,0.5,150,6,10,1200,1800,3000,15,7,15,0,0,0


In [18]:
model.predict(val_df)

array([2])

The new mobile will be of price range 2.

In [19]:
model.feature_importances_

array([0.07394439, 0.00676141, 0.02876979, 0.00666076, 0.02492926,
       0.00659657, 0.03635845, 0.02453614, 0.04003798, 0.02391307,
       0.02953554, 0.05724507, 0.05801654, 0.47648414, 0.02750056,
       0.02887305, 0.03040474, 0.0056038 , 0.00710749, 0.00672124])

In [21]:
pd.DataFrame(model.feature_importances_,x.columns).reset_index()

,index,0
0,battery_power,0.073944
1,blue,0.006761
2,clock_speed,0.028770
3,dual_sim,0.006661
4,fc,0.024929
5,four_g,0.006597
6,int_memory,0.036358
7,m_dep,0.024536
8,mobile_wt,0.040038
9,n_cores,0.023913
